In [3]:
import xml.etree.ElementTree as ET
import shutil
from pathlib import Path

def change_selected_pac(workbook_path, new_pac_value):
    """Change the Selected PAC parameter to a new value"""
    
    # Make a backup copy first
    original = Path(workbook_path)
    backup = original.with_suffix('.twb.backup')
    shutil.copy2(original, backup)
    print(f"Backup created: {backup}")
    
    # Parse the XML
    tree = ET.parse(workbook_path)
    root = tree.getroot()
    
    # Find the Selected PAC parameter and update both alias and value
    for column in root.findall(".//column"):
        if column.get('caption') == 'Selected PAC':
            # Change both the alias and value attributes
            old_alias = column.get('alias')
            old_value = column.get('value')
            
            # Update alias to match the new PAC
            column.set('alias', new_pac_value)
            
            # Update value with proper format
            new_value = f'&quot;PAC - {new_pac_value}&quot;'
            column.set('value', new_value)
            
            print(f"Changed Selected PAC:")
            print(f"  Alias: {old_alias} → {new_pac_value}")
            print(f"  Value: {old_value} → {new_value}")
            
            # Save the modified file
            tree.write(workbook_path, encoding='utf-8', xml_declaration=True)
            print(f"Saved changes to: {workbook_path}")
            return True
    
    print("Selected PAC parameter not found!")
    return False

# Usage
if __name__ == "__main__":
    # Update this path to your actual workbook
    workbook_file = r"walkin\NSW Police Service Assessment Walk-in Auburn;South West Metro.twb"
    
    # Change to Auburn
    #change_selected_pac(workbook_file, "Auburn")

In [4]:
import xml.etree.ElementTree as ET
from pathlib import Path

def debug_parameter_structure(workbook_path):
    """Debug the Selected PAC parameter structure"""
    
    # Parse the XML
    tree = ET.parse(workbook_path)
    root = tree.getroot()
    
    print("=== DEBUGGING SELECTED PAC PARAMETER ===")
    
    # Find all parameters
    found_params = 0
    for column in root.findall(".//column"):
        caption = column.get('caption')
        if caption and 'PAC' in caption:
            found_params += 1
            print(f"\nParameter {found_params}:")
            print(f"  Caption: {caption}")
            
            # Print all attributes
            for attr, value in column.attrib.items():
                print(f"  {attr}: {value}")
            
            # Look for child elements
            print("  Child elements:")
            for child in column:
                print(f"    <{child.tag}> {child.text} (attrs: {child.attrib})")
                
                # Check grandchildren too
                for grandchild in child:
                    print(f"      <{grandchild.tag}> {grandchild.text} (attrs: {grandchild.attrib})")
    
    if found_params == 0:
        print("No PAC parameters found. Let's look for all parameters:")
        
        for column in root.findall(".//column[@caption]"):
            caption = column.get('caption')
            if 'Selected' in caption or 'Parameter' in caption:
                print(f"  Found: {caption}")
    
    print("\n=== END DEBUG ===")

# Usage
if __name__ == "__main__":
    # Update this path to your actual workbook
    workbook_file = r"C:\Users\CL-11\OneDrive - Lonergan Research\Repos\tableau-automation\walkin\NSW Police Service Assessment Walk-in Auburn;South West Metro.twb"
    
    #debug_parameter_structure(workbook_file)

In [5]:
import xml.etree.ElementTree as ET
import shutil
from pathlib import Path

def change_selected_pac(workbook_path, new_pac_value):
    """Change the Selected PAC parameter to a new value"""
    
    # Make a backup copy first
    original = Path(workbook_path)
    backup = original.with_suffix('.twb.backup')
    shutil.copy2(original, backup)
    print(f"Backup created: {backup}")
    
    # Parse the XML
    tree = ET.parse(workbook_path)
    root = tree.getroot()
    
    changes_made = 0
    
    # Find ALL Selected PAC parameters (there are multiple instances)
    for column in root.findall(".//column"):
        if column.get('caption') == 'Selected PAC':
            # Change the alias and value attributes
            old_alias = column.get('alias')
            old_value = column.get('value')
            
            # Update alias to match the new PAC
            column.set('alias', new_pac_value)
            
            # Update value with proper format (remove quotes from new_pac_value)
            new_value = f'"PAC - {new_pac_value}"'
            column.set('value', new_value)
            
            # Also update the calculation formula
            calculation = column.find('calculation')
            if calculation is not None:
                old_formula = calculation.get('formula')
                new_formula = f'"PAC - {new_pac_value}"'
                calculation.set('formula', new_formula)
                print(f"  Updated calculation formula: {old_formula} → {new_formula}")
            
            print(f"Changed Selected PAC instance {changes_made + 1}:")
            print(f"  Alias: {old_alias} → {new_pac_value}")
            print(f"  Value: {old_value} → {new_value}")
            
            changes_made += 1
    
    if changes_made > 0:
        # Save the modified file
        tree.write(workbook_path, encoding='utf-8', xml_declaration=True)
        print(f"Made {changes_made} changes and saved to: {workbook_path}")
        return True
    else:
        print("Selected PAC parameter not found!")
        return False

# Usage
if __name__ == "__main__":
    # Update this path to your actual workbook
    #workbook_file = r"C:\path\to\NSW Police Service Assessment Walk-in Auburn;South West Metro.twb"
    workbook_file = r"C:\Users\CL-11\OneDrive - Lonergan Research\Repos\tableau-automation\walkin\NSW Police Service Assessment Walk-in Auburn;South West Metro.twb"
    
    # Change to Auburn
    #change_selected_pac(workbook_file, "Auburn")

In [6]:
import subprocess
import time
import pyautogui
from pathlib import Path

def export_workbook_to_pdfs(workbook_path, tableau_exe_path, output_dir, pac_name):
    """
    Open Tableau workbook and export each sheet as a separate PDF
    """
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print(f"Opening workbook: {workbook_path}")
    
    # Open Tableau with the workbook
    cmd = f'"{tableau_exe_path}" "{workbook_path}"'
    subprocess.Popen(cmd, shell=True)
    
    # Wait for Tableau to load
    print("Waiting for Tableau to load (20 seconds)...")
    time.sleep(20)
    
    # Click to ensure Tableau window is active
    pyautogui.click(500, 300)
    time.sleep(2)
    
    # Configure pyautogui
    pyautogui.PAUSE = 2
    pyautogui.FAILSAFE = True
    
    # Sheet names to export (adjust these to match your actual sheet names)
    sheets = ["Walk In Page 1", "Walk In Page 2", "Walk In Page 3"]  # Update these!
    
    for i, sheet_name in enumerate(sheets, 1):
        print(f"Exporting sheet {i}: {sheet_name}")
        
        # Click on the sheet tab (you'll need to find the right coordinates)
        # For now, using keyboard shortcuts to navigate between sheets
        if i > 1:
            # Press Ctrl+PageDown to go to next sheet
            pyautogui.hotkey('ctrl', 'pagedown')
            time.sleep(3)
        
        # Export current sheet to PDF
        # File -> Export -> PDF
        pyautogui.hotkey('alt', 'f')  # File menu
        time.sleep(1)
        pyautogui.press('e')  # Export
        time.sleep(1)
        pyautogui.press('p')  # PDF
        time.sleep(3)
        
        # Name the file
        pdf_filename = f"{pac_name}_Page_{i}.pdf"
        output_file = output_path / pdf_filename
        
        # Clear any existing text and type the full path
        pyautogui.hotkey('ctrl', 'a')  # Select all
        time.sleep(1)
        pyautogui.write(str(output_file))
        time.sleep(1)
        pyautogui.press('enter')
        
        # Wait for export to complete
        print(f"Exporting to: {output_file}")
        time.sleep(8)
    
    print("All sheets exported successfully!")
    
    # Close Tableau
    pyautogui.hotkey('alt', 'f4')
    time.sleep(2)

# Combined script - change parameter AND export PDFs
def create_auburn_reports(workbook_path, tableau_exe_path, output_dir):
    """Complete workflow: change parameter and export PDFs"""
    
    # Step 1: Change the parameter to Auburn
    print("Step 1: Changing Selected PAC parameter to Auburn...")
    change_selected_pac(workbook_path, "Auburn")
    
    # Step 2: Export the sheets as PDFs
    print("Step 2: Exporting sheets to PDF...")
    export_workbook_to_pdfs(workbook_path, tableau_exe_path, output_dir, "Auburn")
    
    print("Complete! Auburn reports generated.")

# Usage
if __name__ == "__main__":
    # Update these paths
    workbook_file = r"C:\Users\CL-11\OneDrive - Lonergan Research\Repos\tableau-automation\walkin\NSW Police Service Assessment Walk-in Auburn;South West Metro.twb"
    tableau_executable = r"C:\Program Files\Tableau\Tableau 2023.3\bin\tableau.exe"  # Update version
    output_directory = r"C:\Users\CL-11\OneDrive - Lonergan Research\Repos\tableau-automation\output"
    
    # Run the complete workflow
    #create_auburn_reports(workbook_file, tableau_executable, output_directory)

In [9]:
import subprocess
from pathlib import Path

def export_pdf_command_line(workbook_path, output_dir, pac_name):
    """
    Export PDF using Tableau's command line - no GUI needed
    """
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    pdf_filename = f"{pac_name} - Walkin.pdf"
    output_file = output_path / pdf_filename
    
    # Tableau Desktop command line export
    cmd = [
        r"C:\Program Files\Tableau\Tableau 2024.3\bin\tableau.exe",  # Update path
        "-export-pdf",
        str(output_file),
        str(workbook_path)
    ]
    
    print(f"Exporting PDF via command line...")
    print(f"Command: {' '.join(cmd)}")
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)
        
        if result.returncode == 0:
            print(f"Successfully exported: {output_file}")
            return True
        else:
            print(f"Export failed: {result.stderr}")
            return False
            
    except subprocess.TimeoutExpired:
        print("Export timed out")
        return False
    except Exception as e:
        print(f"Error: {e}")
        return False

def create_auburn_reports_headless(workbook_path, output_dir):
    """Complete workflow without opening Tableau GUI"""
    
    # Step 1: Change the parameter to Auburn
    print("Step 1: Changing Selected PAC parameter to Auburn...")
    change_selected_pac(workbook_path, "Auburn")
    
    # Step 2: Export PDF via command line
    print("Step 2: Exporting PDF via command line...")
    success = export_pdf_command_line(workbook_path, output_dir, "Auburn")
    
    if success:
        print("Complete! Auburn - Walkin.pdf generated without opening Tableau GUI.")
    else:
        print("Command line export failed. Falling back to GUI method...")
        # Could fall back to the GUI method here
        
    return success

# Usage
if __name__ == "__main__":
    workbook_file = r"C:\Users\CL-11\OneDrive - Lonergan Research\Repos\tableau-automation\walkin\NSW Police Service Assessment Walk-in Auburn;South West Metro.twb"
    output_directory = r"C:\Users\CL-11\OneDrive - Lonergan Research\Repos\tableau-automation\output"
    
    create_auburn_reports_headless(workbook_file, output_directory)

Step 1: Changing Selected PAC parameter to Auburn...
Backup created: C:\Users\CL-11\OneDrive - Lonergan Research\Repos\tableau-automation\walkin\NSW Police Service Assessment Walk-in Auburn;South West Metro.twb.backup
  Updated calculation formula: "PAC - Auburn" → "PAC - Auburn"
Changed Selected PAC instance 1:
  Alias: Auburn → Auburn
  Value: "PAC - Auburn" → "PAC - Auburn"
  Updated calculation formula: "PAC - Auburn" → "PAC - Auburn"
Changed Selected PAC instance 2:
  Alias: Auburn → Auburn
  Value: "PAC - Auburn" → "PAC - Auburn"
  Updated calculation formula: "PAC - Auburn" → "PAC - Auburn"
Changed Selected PAC instance 3:
  Alias: Auburn → Auburn
  Value: "PAC - Auburn" → "PAC - Auburn"
  Updated calculation formula: "PAC - Auburn" → "PAC - Auburn"
Changed Selected PAC instance 4:
  Alias: Auburn → Auburn
  Value: "PAC - Auburn" → "PAC - Auburn"
  Updated calculation formula: "PAC - Auburn" → "PAC - Auburn"
Changed Selected PAC instance 5:
  Alias: Auburn → Auburn
  Value: "PAC

In [10]:
import pyautogui
print(pyautogui.position())

Point(x=1066, y=581)


In [24]:
print("Selecting all 4 pages...")
pyautogui.click(946, 1397)  # Click Page 1 tab
time.sleep(0.5)
pyautogui.keyDown('shift')  # Hold Shift
pyautogui.click(1165, 1402)  # Click Page 4 tab while holding Shift
pyautogui.keyUp('shift')  # Release Shift
time.sleep(1)
pyautogui.click(22, 40)
time.sleep(1)

Selecting all 4 pages...


In [ ]:
pyautogui.click(63, 438) 
time.sleep(1)

Selecting all 4 pages...


In [25]:
# select selected sheets
pyautogui.click(218, 720) 
time.sleep(1)

In [ ]:

# Click on "View PDF after printing" checkbox  




# Click OK button
pyautogui.click(667, 812)
time.sleep(3)

# Type the PDF filename
pyautogui.write(f"Auburn - Walkin.pdf")

pyautogui.click(1533, 1303)
time.sleep(3)